## Let's visualize the Kumon Schools in Toronto ##

In [5]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import math

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


In [6]:
CLIENT_ID = 'VBSBYNY2VBFL0HR4KT0ACIIXSHLZDSFT3HIRAWSW5WO4OXBW' # your Foursquare ID
CLIENT_SECRET = 'UGRKTBG241NH0K2ERHHCY3S1I3T3W1YOANUYBQW5ANU13WQH' # your Foursquare Secret
VERSION = '20180605'
LIMIT = 30

In [7]:
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode('Toronto, ON')
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [90]:
curDir = os.getcwd()
file_path = curDir + '/Data_files/'
df_Toronto = pd.read_csv(file_path + 'df_toronto_out.csv')

df_Kumon  = pd.read_csv(file_path + 'kumon_out.csv')

# Data cleanse to remove outliers in the nearby cities - mississauga, richmond hill and vaughn
df_Toronto=df_Toronto[~df_Toronto['Borough'].isin(['Mississauga',"Queen's Park",])]
df_Kumon=df_Kumon[~df_Kumon['city'].isin(['Mississauga','Vaughan','Richmond Hill'])]
df_Kumon=df_Kumon[~df_Kumon.id.isin(['510fd103e4b0d7e127541659','4fcfc762e4b0286d5c829200'])]
print(df_Toronto.shape, df_Kumon.shape )

(101, 7) (22, 16)


In [93]:
df_Toronto['Child age 0-14 Pop Size'] = pd.to_numeric(df_Toronto['Child age 0-14 Pop Size'], errors='coerce').fillna(0).astype(np.int64)

In [95]:
df_Toronto.tail()

,Unnamed: 0,Postcode,Borough,Neighbourhood,Child age 0-14 Pop Size,Latitude,Longitude
98,98,M9N,York,Weston,4345,43.706876,-79.518188
99,99,M9P,Etobicoke,Westmount,3140,43.696319,-79.532242
100,100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",6075,43.688905,-79.554724
101,101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",10730,43.739416,-79.588437
102,102,M9W,Etobicoke,Northwest,6595,43.706748,-79.594054


In [96]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.653963 -79.387207


In [100]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add a red circle marker to represent Toronto FSA area
for lat, lng, Borough, Postcode, pop_size in zip(df_Toronto['Latitude'], df_Toronto['Longitude'], df_Toronto['Borough'],df_Toronto['Postcode'],df_Toronto['Child age 0-14 Pop Size']):
    label = Borough + ' (' + Postcode + ')'
    label = folium.Popup(label, parse_html=True)
    rd_size = int(pop_size/1000)
    folium.CircleMarker(
        [lat, lng],
        radius=rd_size,
        popup=label,
        color='red',
        fill=True,
        fill_color = 'white',
        fill_opacity=0.7,
        parse_html=False
    ).add_to(map_toronto)  
#map_toronto

In [101]:
# add the Schools as blue circle markers
#map_kumon = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, label in zip(df_Kumon.lat, df_Kumon.lng, df_Kumon.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=6,
        color='black',
        popup=label,
        fill = True,
        fill_color='black',
        fill_opacity=0.6,
        parse_html=False
    ).add_to(map_toronto)

# display map
map_toronto

In [4]:
!pip install haversine

  Running setup.py bdist_wheel for haversine ... done
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/f5/0d/3d/26a6615a5297b894e02dc422fcf887b3052309b78b8fdee08a
Successfully built haversine
distributed 1.21.8 requires msgpack, which is not installed.


In [5]:
from haversine import haversine

In [10]:
lyon = (45.7597, 4.8422) # (lat, lon)
paris = (48.8567, 2.3508)

distance = round(haversine(lyon, paris),1)
distance

392.2

In [104]:
''' Search for nearby Venues by location '''
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [196]:
west_toronto_venues = getNearbyVenues(names=df_Toronto['Borough'],
                                   latitudes=df_Toronto['Latitude'],
                                   longitudes=df_Toronto['Longitude']
                                  )

Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
East York
East York
East Toronto
East York
East York
East York
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
North York
York
York
Downtown Toronto
Wes

In [199]:
df_Toronto['Borough'].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       "Queen's Park", 'Mississauga', 'Etobicoke'], dtype=object)

In [201]:
df_Toronto.shape

(103, 7)